# Custom LLLM Agent

This notebook goes through how to create your own custom LLM agent.

An LLM agent consists of three parts:
    
    - Tools: The tools the agent has available to use.
    - LLMChain: The LLMChain that produces the text that is parsed in a certain way to determine which action to take.
    - OutputParser: This determines how to parse the LLMOutput into 
        
        
In this notebook we walk through how to create a custom LLM agent.

In [1]:
import sys
sys.path.append("../../../../../lang-chat/backend/")
import os
os.environ["LANGCHAIN_HANDLER"] = "langchain"

In [2]:
from app.toolkits.open_api.toolkit import OpenAPIToolkit
urls = [
    "https://www.klarna.com/.well-known/ai-plugin.json"
]
toolkits = list(map(OpenAPIToolkit.from_plugin_url, urls))
namespaces = "\n\n".join([tk.get_typescript_namespace() for tk in toolkits])
#print(namespaces)

In [3]:
from langchain.agents import Tool, AgentExecutor
from langchain.agents.agent import LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain import OpenAI, SerpAPIWrapper, LLMChain

In [4]:
template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

If you want use a tool you should write valid TypeScript code to call it. Please put all parameters and their values on their own line. If you use strings, they should be wrapped in double quotes not single.

For your response, you have two options:

If you do not need to use any tools, you can just respond directly to the user by using the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Final Answer: your response to user here

If you do need to use tools, you should use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: ```typescript
...
``` 
Observation: the result of the action
... (this Thought/Action/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question.

When repsonding in your final answer to the user, they have NO knowledge of any intermediate steps. So if there is any intermediate knowledge there you want them to know, you should make sure to return it as part of your final answer.
Begin!

{chat_history}
Question: {input}
Thought: {agent_scratchpad}"""

In [15]:
shopify = toolkits[0]

In [16]:
class CustomPromptTemplate(StringPromptTemplate):
    
    def format(self, **kwargs) -> str:
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        kwargs["agent_scratchpad"] = thoughts
        kwargs["tools"] = shopify.get_typescript_namespace()
        return template.format(**kwargs)

In [17]:
prompt = CustomPromptTemplate(input_variables=["input", "intermediate_steps", "chat_history"])

In [18]:
llm = OpenAI(temperature=0, max_tokens=-1)

In [19]:
query = "what are some cheap pokemon t-shirts?"

In [28]:
FINAL_ANSWER_ACTION = "Final Answer:"
import json
import json5
from langchain.schema import AgentAction, AgentFinish
import re
class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output):
        if FINAL_ANSWER_ACTION in llm_output:
            return AgentFinish(
                return_values={"output": llm_output.split(FINAL_ANSWER_ACTION)[-1].strip()},
                log=llm_output,
            )
        parsed = llm_output.split("Action:")[1].split("```typescript")[1].split("```")[0].strip()
        typescript_str = parsed.split("(")[1].split(")")[0]
        tool_input = json.dumps(json5.loads(typescript_str))
        return AgentAction(tool=parsed.split("(")[0], tool_input=tool_input, log=llm_output)

In [29]:
output_parser = CustomOutputParser()

In [30]:
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [31]:
agent = LLMSingleActionAgent(llm_chain=llm_chain, output_parser=output_parser, stop=["\nObservation:"])

In [32]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history")

In [33]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=shopify.get_tools(), verbose=True, memory=memory)

In [34]:
agent_executor.run(input="hi, i am bob")



> Entering new AgentExecutor chain...
 Bob is looking for help
Action: ```typescript
KlarnaProducts.productsUsingGET({
	q: "help",
	size: 10,
	budget: null
})
```

Observation:{"products":[{"name":"Help Dorman 77573 Interior Door Handle for Specific Cadillac Models Blue;","url":"https://www.klarna.com/us/shopping/pl/cl709/3204473989/Building-Materials/Help-Dorman-77573-Interior-Door-Handle-for-Specific-Cadillac-Models-Blue/?utm_source=openai","price":"$16.34","attributes":[]},{"name":"Creatine Carnage Warfare Creatine Powder to Help Build Lean","url":"https://www.klarna.com/us/shopping/pl/cl566/3203978673/Nutrition-Supplements/Creatine-Carnage-Warfare-Creatine-Powder-to-Help-Build-Lean/?utm_source=openai","price":"$45.99","attributes":[]},{"name":"Help Help The Globolinks (DVD)","url":"https://www.klarna.com/us/shopping/pl/cl801/886794/Movies/Help-Help-The-Globolinks-%28DVD%29/?utm_source=openai","price":"$24.51","attributes":["Genre:Unclassified"]},{"name":"Help Me, Help You [LP] VI

'Hi Bob, here are some products that might help you: Help Dorman 77573 Interior Door Handle for Specific Cadillac Models Blue;, Creatine Carnage Warfare Creatine Powder to Help Build Lean, Help Help The Globolinks (DVD), Help Me, Help You [LP] VINYL, Sylar Help!, Suzanne Ciani - Help, Help, The Globolinks! [VINYL], help Wanted [VINYL], Help! [Blu-ray] [1965], Raconteurs Help Us Stranger (Vinyl), The Beatles - Help! [VINYL].'

In [ ]:
agent_executor.run("whats my name?")

In [ ]:
agent_executor.run(query)

In [ ]:
agent_executor.run("hi")